## import packages, define variables

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd # for creation of data frames
import numpy as np #
import seaborn as sns
import nibabel as nib # for loading niftis etc. as arrays
import matplotlib.pyplot as plt # for plotting data
#the output of plotting commands is displayed inline, directly below the code cell that produced it
%matplotlib inline 
import os,glob
from nilearn import plotting, input_data, image #for plotting & working with niftis

import pathlib
import pyxnat

###################### update path!! #####################################
base_path = '/root_dir/' ####
##########################################################################

#import qBOLD functions
#import sys
#sys.path.append(os.path.join(base_path + 'scripts/'))
#import mqBOLD_functions as qB


### define FSL directories ######################################################
os.environ["FSLDIR"]='/usr/share/fsl/5.0'
os.environ["FSLOUTPUTTYPE"]='NIFTI_GZ'
os.environ["FSLTCLSH"]='/usr/bin/tclsh'
os.environ["FSLWISH"]='/usr/bin/wish'
os.environ["FSLMULTIFILEQUIT"]="True"
os.environ["LD_LIBRARY_PATH"]='/usr/share/fsl/5.0:/usr/lib/fsl/5.0'
#################################################################################

In [ ]:
data_dir = base_path + 'data/'
derivatives_dir = data_dir + 'derivatives'
raw_dir = data_dir + 'rawdata'
results_dir = base_path + 'results'

sns.set_style("whitegrid")

## N=40
sids = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68]
## N=30
sids_30 = [19, 20, 21, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55]

df_participants = pd.read_csv(data_dir + '/participants.tsv',sep='\t')

coords=(-15, 0, 15, 30, 45, 60, 75, 90)

### Fig. 1C: median across all subjects per paramater map, control condition, MNI space

In [ ]:
import matplotlib as mpl

cond='rest'

parameters=['T2map', 'T2Smap', 'cbv', 'cbf', 'R2prime', 'oef', 'cmro2']
parameters=['cmro2']
MNI_2mm = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm.nii.gz')
MNI_2mm_brain = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz')
! bet {MNI_2mm} {MNI_2mm_brain}
Br_MNI = nib.load(os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz'))

for par in parameters:
    #par_map = np.zeros((len(sids), len_mask))
    par_map = np.zeros(shape=(91,109,91,len(sids))) #shape = MNI, 4D = length(sids)
    
    for i, ID in enumerate(sids_30): #loop over subjects
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        
        sub_dir = os.path.join(data_dir, sub)

        dir_anat = os.path.join(sub_dir, 'anat')
        dir_func = os.path.join(sub_dir, 'func')
        dir_perf = os.path.join(sub_dir, 'perf')

        dir_anat_deriv = os.path.join(derivatives_dir, sub, 'anat')
        dir_func_deriv = os.path.join(derivatives_dir, sub, 'func')
        dir_perf_deriv = os.path.join(derivatives_dir, sub, 'perf')
        dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')

        if par != 'T2map' and par != 'cmro2':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_'+ par +'.nii.gz')
        if par == 'T2map':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_space-MNI152_'+ par +'.nii.gz')
        if par == 'cmro2':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_desc-orig_'+ par +'.nii.gz')

        par_values = np.array(nib.load(par_nii).dataobj)
        #par_map[i, :] = masker.fit_transform(par_nii)
        par_map[:, :, :, i] = par_values
        if par == 'cbf': ##upscale
            par_map[:, :, :, i] = par_values/0.75
            

    ## median per voxel, across subjects
    par_map_median = np.nanmedian(par_map, axis=3)
    
    ##turn into nifti
    par_map_nii = nib.Nifti1Image(par_map_median, affine=Br_MNI.affine)
    ##save nifti
    nib.save(par_map_nii, results_dir+ '/N' + str(len(sids)) + '_cond-'+cond + '_median_'+par+'.nii.gz')
    
    ## plot nifti
    if par == 'T2map':
        min_thr=10
        max_thr= 150
        colormap= 'Greys_r'
        thr=10
        bound = [10, 40, 70, 100, 130]
    if par == 'T2Smap':
        min_thr=10
        max_thr= 100
        colormap= 'Greys_r'
        thr=10
        bound = [10, 40, 70, 100]
    if par == 'cbf':
        min_thr=5
        max_thr= 70
        colormap= 'jet'
        thr=5    
        bound = [5, 25, 45, 65]
    if par == 'cbv':
        min_thr=0.1
        max_thr= 10
        colormap= 'jet'
        thr=0.1
        bound = [0.1, 2, 4, 6, 8, 10]
    if par == 'R2prime':
        min_thr=2
        max_thr= 10
        colormap= 'jet'
        thr=2
        bound = [2, 4, 6, 8, 10]
    if par == 'oef':
        min_thr=0.1
        max_thr= 0.9
        colormap= 'jet'
        thr=0.1
        bound = [0.1, 0.3, 0.5, 0.7, 0.9]
    if par == 'cmro2':
        min_thr=10
        max_thr= 300
        colormap= 'jet'
        thr=10
        bound = [10, 50, 100, 150, 200, 250,300]
            
    ## plot median image
    plotting.plot_img(par_map_nii, threshold=thr, display_mode='z', vmin=min_thr, vmax=max_thr, cmap=colormap, colorbar=True, cut_coords=coords)
    fig.figure.savefig(results_dir+ '/' + par + '_median.png', format='png', dpi=300)
    
    
    ## plot colorbar separately
    ## draw stand-alone colourbar
    fig, ax = plt.subplots(figsize=(3.5, 0.5))
    fig.subplots_adjust(bottom=0.5)

    #cmap = mpl.cm.Greys_r
    bounds = bound
    norm = mpl.colors.Normalize(vmin=min_thr, vmax=max_thr)

    cbar=fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colormap), 
                 cax=ax, ticks=bounds, orientation='horizontal', label=par)
    cbar.ax.tick_params(labelsize=15)
    plt.tight_layout()
    
    fig.figure.savefig(results_dir + '/' + par + '_colorbar.png', format='png', dpi=300)

## median only within GM

In [ ]:
import matplotlib as mpl

#cond='rest'
cond='control'

parameters=['T2map', 'T2Smap', 'cbv', 'cbf', 'R2prime', 'oef', 'cmro2']
#parameters=['cmro2']

MNI_2mm = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm.nii.gz')
MNI_2mm_brain = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz')
! bet {MNI_2mm} {MNI_2mm_brain}
Br_MNI = nib.load(os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz'))

## brain mask_ R2/CBV mask or only GM mask
#whole_brain_mask = os.path.join( derivatives_dir, 'task-all_space-MNI152_res-2_SNR_YEO_group_mask.nii.gz')
whole_brain_mask = os.path.join(derivatives_dir, 'N40_CBV_R2s_mask.nii.gz')
masker_GM = input_data.NiftiMasker(mask_img=whole_brain_mask)
mask_img = masker_GM.fit_transform(MNI_2mm_brain)
len_mask = len(masker_GM.fit_transform(whole_brain_mask).flatten())


for par in parameters:
    par_map = np.zeros((len(sids), len_mask))
    
    for i, ID in enumerate(sids_30): #loop over subjects
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        
        sub_dir = os.path.join(data_dir, sub)

        dir_anat = os.path.join(sub_dir, 'anat')
        dir_func = os.path.join(sub_dir, 'func')
        dir_perf = os.path.join(sub_dir, 'perf')

        dir_anat_deriv = os.path.join(derivatives_dir, sub, 'anat')
        dir_func_deriv = os.path.join(derivatives_dir, sub, 'func')
        dir_perf_deriv = os.path.join(derivatives_dir, sub, 'perf')
        dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')

        par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_'+ par +'.nii.gz')
        if par != 'T2map' and par != 'cmro2':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_'+ par +'.nii.gz')
        if par == 'T2map':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_space-MNI152_'+ par +'.nii.gz')
        if par == 'cmro2':
            par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_desc-orig_'+ par +'.nii.gz')

        par_values = np.array(nib.load(par_nii).dataobj)
        par_values = masker_GM.fit_transform(par_nii)
        if par == 'cbf': ##upscale
            par_values = par_values/0.75        #par_values = np.array(nib.load(par_nii).dataobj)
        
        par_map[i, :] = par_values

            

    ## median per voxel, across subjects
    par_map_median = np.nanmedian(par_map, axis=0)
    
    ## calculate across-voxel mean + variance
    mean= np.nanmean(par_map_median)
    print('mean across voxels: ' + str(mean))
    std= np.nanstd(par_map_median)
    print('std across voxels: ' + str(std))
    
    ##turn into nifti
    par_map_nii = masker_GM.inverse_transform(par_map_median)
    ##save nifti
    nib.save(par_map_nii, results_dir+ '/N' + str(len(sids)) + '_cond-'+cond + '_median_'+par+'_lowres_GM.nii.gz')
    
    ## plot nifti
    if par == 'T2map':
        min_thr=10
        max_thr= 150
        colormap= 'Greys_r'
        thr=10
        bound = [10, 40, 70, 100, 130]
    if par == 'T2Smap':
        min_thr=10
        max_thr= 100
        colormap= 'Greys_r'
        thr=10
        bound = [10, 40, 70, 100]
    if par == 'cbf':
        min_thr=5
        max_thr= 70
        colormap= 'jet'
        thr=5    
        bound = [5, 25, 45, 65]
    if par == 'cbv':
        min_thr=0.1
        max_thr= 10
        colormap= 'jet'
        thr=0.1
        bound = [0.1, 2, 4, 6, 8, 10]
    if par == 'R2prime':
        min_thr=2
        max_thr= 10
        colormap= 'jet'
        thr=2
        bound = [2, 4, 6, 8, 10]
    if par == 'oef':
        min_thr=0.1
        max_thr= 0.9
        colormap= 'jet'
        thr=0.1
        bound = [0.1, 0.3, 0.5, 0.7, 0.9]
    if par == 'cmro2':
        min_thr=10
        max_thr= 300
        colormap= 'jet'
        thr=10
        bound = [10, 50, 100, 150, 200, 250,300]

    ## plot median image
    #fig, ax = plt.subplots(figsize=(3.5, 0.5))
    fig = plotting.plot_img(par_map_nii, bg_img = MNI_2mm_brain, threshold=thr, display_mode='z', vmin=min_thr, vmax=max_thr, cmap=colormap, colorbar=True, cut_coords=coords)
    fig.savefig(results_dir+ '/' + par + '_median.png',  dpi=300)
    
    ## plot colorbar separately
    ## draw stand-alone colourbar
    fig, ax = plt.subplots(figsize=(3.5, 0.5))
    fig.subplots_adjust(bottom=0.5)

    #cmap = mpl.cm.Greys_r
    bounds = bound
    norm = mpl.colors.Normalize(vmin=min_thr, vmax=max_thr)

    cbar=fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colormap), 
                 cax=ax, ticks=bounds, orientation='horizontal', label=par)
    cbar.ax.tick_params(labelsize=15)
    plt.tight_layout()
    
    fig.savefig(results_dir + '/' + par + '_colorbar.png', format='png', dpi=300)

## CBV median, calc and control conditions

In [ ]:
import matplotlib as mpl

conds=['control', 'calc']

parameters=['cbv']
MNI_2mm = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm.nii.gz')
MNI_2mm_brain = os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz')
! bet {MNI_2mm} {MNI_2mm_brain}
Br_MNI = nib.load(os.path.join(data_dir, 'derivatives', 'MNI152_T1_2mm_brain.nii.gz'))

for cond in conds:
    #par_map = np.zeros((len(sids), len_mask))
    par_map = np.zeros(shape=(91,109,91,len(sids))) #shape = MNI, 4D = length(sids)
    
    for i, ID in enumerate(sids): #loop over subjects
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)
        
        sub_dir = os.path.join(data_dir, sub)

        dir_anat = os.path.join(sub_dir, 'anat')
        dir_func = os.path.join(sub_dir, 'func')
        dir_perf = os.path.join(sub_dir, 'perf')

        dir_anat_deriv = os.path.join(derivatives_dir, sub, 'anat')
        dir_func_deriv = os.path.join(derivatives_dir, sub, 'func')
        dir_perf_deriv = os.path.join(derivatives_dir, sub, 'perf')
        dir_qmri_deriv = os.path.join(derivatives_dir, sub, 'qmri')

        par_nii =  os.path.join(dir_qmri_deriv, sub + '_task-'+cond+'_space-MNI152_cbv.nii.gz')

            
        par_values = np.array(nib.load(par_nii).dataobj)
        #par_map[i, :] = masker.fit_transform(par_nii)
        par_map[:, :, :, i] = par_values


    ## median per voxel, across subjects
    par_map_median = np.nanmedian(par_map, axis=3)
    
    ##turn into nifti
    par_map_nii = nib.Nifti1Image(par_map_median, affine=Br_MNI.affine)
    ##save nifti
    nib.save(par_map_nii, results_dir+ '/N' + str(len(sids)) + '_cond-'+cond + '_space-MNI152_median_cbv.nii.gz')
    
    ## plot nifti
    min_thr=0.1
    max_thr= 10
    colormap= 'jet'
    thr=0.1
    bound = [0.1, 2, 4, 6, 8, 10]

    ## plot median image
    plotting.plot_img(par_map_nii, threshold=thr, display_mode='z', vmin=min_thr, vmax=max_thr, cmap=colormap, colorbar=True, cut_coords=coords)
    #fig.figure.savefig(results_dir+ '/' + par + '_median.png', format='png', dpi=300)
    
    
    ## plot colorbar separately
    ## draw stand-alone colourbar
    fig, ax = plt.subplots(figsize=(3.5, 0.5))
    fig.subplots_adjust(bottom=0.5)

    #cmap = mpl.cm.Greys_r
    bounds = bound
    norm = mpl.colors.Normalize(vmin=min_thr, vmax=max_thr)

    cbar=fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colormap), 
                 cax=ax, ticks=bounds, orientation='horizontal', label='cbv')
    cbar.ax.tick_params(labelsize=15)
    plt.tight_layout()
    
    #fig.figure.savefig(results_dir + '/' + par + '_colorbar.png', format='png', dpi=300)

### Fig. 1C: boxplots, all conditions, subject space

In [ ]:
parameters=['T2map', 'T2Smap', 'cbv', 'cbf', 'R2prime', 'oef', 'cmro2']
fontsize=24
sns.set_style("whitegrid")

## load thresholded native-space parameter maps for each condition, average across GM and plot as boxplots
for par in parameters:
    par_val = np.zeros((len(sids), 4))
    par_val[:] = np.nan ##empy nan array
    par_GM_control = np.zeros((len(sids), 1))
    for i, ID in enumerate(sids): #loop over subjects
        sub = "sub-p{:03d}".format(ID) # subject id, eg 'p021'
        print(sub)

        sub_dir = os.path.join(data_dir, sub)

        dir_deriv = os.path.join(derivatives_dir, sub)

        if ID < 56:
            conds = ['rest', 'control', 'mem', 'calc']
        if ID > 55:
            conds = ['control', 'calc']        
        if par == 'T2map':
            conds = ['control']
        if par == 'cbv' and ID < 56:
            conds = ['control']
        if par == 'cbv' and ID >55:
            conds = ['control', 'calc']
            
        for k, cond in enumerate(conds):
            ## load parameter map
            if par != 'T2map' and par != 'cmro2':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_'+ par +'_qBmasked.nii.gz')
            if par == 'T2map':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_space-T2_'+ par +'_qBmasked.nii.gz')
            if par == 'cmro2' and cond == 'calc':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_desc-CBV_'+ par +'_qBmasked.nii.gz')
            if par == 'cmro2' and cond != 'calc':
                    par_nii =  os.path.join(dir_deriv, 'qmri', sub + '_task-'+cond+'_space-T2_desc-orig_'+ par +'_qBmasked.nii.gz')
                    
            par_arr = np.array(nib.load(par_nii).dataobj)
            subj_median = np.nanmedian(par_arr[par_arr>0]) ##only values bigger than zero
            subj_mean = np.nanmean(par_arr[par_arr>0]) ##only values bigger than zero
            if par == 'cbf': ##upscale by 25%
                subj_median = subj_median/0.75
                subj_mean = subj_mean/0.75
                
            par_val[i, k] = subj_mean           
            
            ## control mean values  
            if cond == 'control':
                par_GM_control[i] = subj_mean
    
    ## mean per subject, sd across subjects, in ctrl       
    print('mean CTRL = ' + par + ' ' + str(np.nanmean(par_GM_control)))
    print('std CTRL = ' + par + ' '+ str(np.nanstd(par_GM_control)))
    
    ## mean per subject, sd across subjects, across conditions, as in barplots      
    print('mean all conds = ' + par + ' ' + str(np.nanmean(par_val.flatten())))
    print('std all conds = ' + par + ' '+ str(np.nanstd(par_val.flatten())))
    
    if par == 'T2map':
        label= 'T2 [ms]'
    if par == 'T2Smap':
        label= 'T2* [ms]'
    if par == 'cbv':
        label= 'CBV [%]'        
    if par == 'cbf':
        label= 'cbf [ml/100g/min]'
    if par == 'R2prime':
        label= 'R2p [1/s]'
    if par == 'oef':
        label= 'OEF [ratio]'
    if par == 'cmro2':
        label= 'CMRO2 [μmol/100g/min]'
        
    ## plot boxplot
    fig, ax = plt.subplots(1, figsize=(2,6))
    sns.boxplot(y=par_val.flatten(), color='lightgray', boxprops=dict(alpha=.8), ax=ax, orient="v")
    sns.swarmplot(y=par_val.flatten(), color=".25", ax=ax, orient="v")
    ax.set_xlabel(label, fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.tight_layout()
    plt.show()